In [4]:
%%bash
apt-get install openjdk-8-jdk-headless -qq > /dev/null

wget -q https://downloads.apache.org/spark/spark-2.4.7/spark-2.4.7-bin-hadoop2.7.tgz
tar xf spark-2.4.7-bin-hadoop2.7.tgz

pip install findspark

tar: spark-2.4.7-bin-hadoop2.7.tgz: Cannot open: No such file or directory
tar: Error is not recoverable: exiting now


In [2]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.7-bin-hadoop2.7"

In [3]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

Exception: ignored

In [ ]:
data = spark.read.csv('DS_2019_public.csv',inferSchema=True,sep=",", header=True,)
df = data.select('DIVISION', 'REPORTABLE_DOMAIN', 'TOTALDOLCOL', 'TOTALBTUCOL', 'CELLAR', 'NWEIGHT', 'DOLLAREL', 'TOTUCSQFT','Climate_Region_Pub')
cols = df.columns
data.show(10)
df.printSchema()


+------------------+--------+-----------------+--------+-----------+--------+---------+-----------+-----------+-----------+------+-----------+--------+--------+-----+--------+--------+-----------+---------+--------+--------+--------+--------+-------+------+-----------+--------+-----------+-----+--------+--------+--------+--------+---------+------+-----+--------+--------+--------+--------+--------+--------+------+-----------+--------+--------+-------+-----------+--------+--------+------------+---------+---------+---------+-------+----------+--------+--------+-------+--------+--------+--------+---------+------+-------+-------+--------+--------+--------+---------+--------+--------+--------+---------+--------+-------+------------+---------+-----------+-----------+--------+--------+----+------+-------+--------+-----+----+-------+-------+-----------+--------+-------+--------+-----------+---------+--------+---------+-----------+--------+-----+--------+---------+-----+-----+--------+----------

In [ ]:
cols=raw_data.columns
cols.remove("Outcome")
# Let us import the vector assembler
from pyspark.ml.feature import VectorAssembler
assembler = VectorAssembler(inputCols=cols,outputCol="features")
# Now let us use the transform method to transform our dataset
raw_data=assembler.transform(raw_data)
raw_data.select("features").show(truncate=False)

In [ ]:
from pyspark.ml.feature import OneHotEncoderEstimator, StringIndexer, VectorAssembler

categoricalColumns = []
stages = []
for categoricalCol in categoricalColumns:
    stringIndexer = StringIndexer(inputCol = categoricalCol, outputCol = categoricalCol + 'Index')
    encoder = OneHotEncoderEstimator(inputCols=[stringIndexer.getOutputCol()], outputCols=[categoricalCol + "classVec"])
    stages += [stringIndexer, encoder]

label_stringIdx = StringIndexer(inputCol = 'Climate_Region_Pub', outputCol = 'label')
stages += [label_stringIdx]
numericCols = ['DIVISION', 'REPORTABLE_DOMAIN', 'TOTALDOLCOL', 'TOTALBTUCOL', 'CELLAR','NWEIGHT', 'DOLLAREL', 'TOTUCSQFT','Climate_Region_Pub']
assemblerInputs = [c + "classVec" for c in categoricalColumns] + numericCols
assembler = VectorAssembler(inputCols=assemblerInputs, outputCol="features")
stages += [assembler]

In [ ]:
from pyspark.ml import Pipeline

pipeline = Pipeline(stages = stages)
pipelineModel = pipeline.fit(df)
df = pipelineModel.transform(df)
selectedCols = ['label', 'features'] + cols
df = df.select(selectedCols)
df.printSchema()

root
 |-- label: double (nullable = false)
 |-- features: vector (nullable = true)
 |-- DIVISION: integer (nullable = true)
 |-- REPORTABLE_DOMAIN: integer (nullable = true)
 |-- TOTALDOLCOL: integer (nullable = true)
 |-- TOTALBTUCOL: integer (nullable = true)
 |-- CELLAR: integer (nullable = true)
 |-- NWEIGHT: double (nullable = true)
 |-- DOLLAREL: integer (nullable = true)
 |-- TOTUCSQFT: integer (nullable = true)
 |-- Climate_Region_Pub: integer (nullable = true)



In [ ]:
train, test = df.randomSplit([0.7, 0.3], seed = 100)
print("Training Dataset Count: " + str(train.count()))
print("Test Dataset Count: " + str(test.count()))

Training Dataset Count: 7679
Test Dataset Count: 3196


Логистическая регрессия


In [ ]:
from pyspark.ml.classification import LogisticRegression
lr = LogisticRegression(featuresCol = 'features', labelCol = 'label', maxIter=10)
lrModel = lr.fit(train)

In [ ]:
predictions = lrModel.transform(test)
predictions.select('Climate_Region_Pub','DIVISION', 'REPORTABLE_DOMAIN', 'TOTALDOLCOL', 'TOTALBTUCOL', 'CELLAR', 'NWEIGHT', 'DOLLAREL', 'TOTUCSQFT', 'label', 'rawPrediction', 'prediction', 'probability').show(100000)

In [ ]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator
evaluator = BinaryClassificationEvaluator()
print('Test Area Under ROC', evaluator.evaluate(predictions))

Test Area Under ROC 0.6712408473179982


Random Forest

In [ ]:
from pyspark.ml.classification import RandomForestClassifier
rf = RandomForestClassifier(featuresCol = 'features', labelCol = 'label')
rfModel = rf.fit(train)
predictions = rfModel.transform(test)
predictions.select('Climate_Region_Pub','DIVISION', 'REPORTABLE_DOMAIN', 'TOTALDOLCOL', 'TOTALBTUCOL', 'CELLAR', 'NWEIGHT', 'DOLLAREL', 'TOTUCSQFT', 'label', 'rawPrediction', 'prediction', 'probability').show(100000)

In [ ]:
evaluator = BinaryClassificationEvaluator()
print("Test Area Under ROC: " + str(evaluator.evaluate(predictions, {evaluator.metricName: "areaUnderROC"})))

Test Area Under ROC: 0.7601109910178194


1.	Найдите среднюю стоимость жилья (sale_price) и выведите новую таблицу, содержащую две колонки – стоимость жилья и отклонение стоимости от среднего значения.

In [ ]:
import pyspark.sql.functions as f
avg_sales = data.groupBy().mean('sale_price').collect()[0][0]
avg_sales

Decimal('4064219.7668')

In [ ]:
data.select(data['sale_price'], (data['sale_price'] - 4064219.7668).alias('sale_price deviation')).show()

+----------+--------------------+
|sale_price|sale_price deviation|
+----------+--------------------+
| 499401179|    4.953369592332E8|
| 345000000|    3.409357802332E8|
| 340000000|    3.359357802332E8|
| 276947000|    2.728827802332E8|
| 202500000|    1.984357802332E8|
| 185445000|    1.813807802332E8|
| 171000000|    1.669357802332E8|
| 169000000|    1.649357802332E8|
| 165000000|    1.609357802332E8|
| 161000000|    1.569357802332E8|
| 160000000|    1.559357802332E8|
| 158000000|    1.539357802332E8|
| 154250000|    1.501857802332E8|
| 138106368|    1.340421482332E8|
| 135212000|    1.311477802332E8|
| 135000000|    1.309357802332E8|
| 132400000|    1.283357802332E8|
| 125000000|    1.209357802332E8|
| 124000000|    1.199357802332E8|
| 121313223|    1.172490032332E8|
+----------+--------------------+
only showing top 20 rows



2.	Найдите среднюю стоимость жилья (sale_price) для каждого района.

In [ ]:
data.groupBy('neighborhood').avg('sale_price').show()


+-------------------+---------------+
|       neighborhood|avg(sale_price)|
+-------------------+---------------+
|     FLATBUSH-NORTH|   6371426.1061|
|      CYPRESS HILLS|   4235258.6735|
|        BENSONHURST|   2612471.4104|
|       BOROUGH PARK|   2946290.4185|
|    WINDSOR TERRACE|   1854183.9533|
|OCEAN PARKWAY-NORTH|   2919510.3169|
|          NAVY YARD|   8065875.3889|
|            SEAGATE|   2141875.0000|
|       CLINTON HILL|   3627488.9714|
|   PROSPECT HEIGHTS|   2764040.9777|
| DOWNTOWN-METROTECH|  22016642.6627|
|      FLATBUSH-EAST|   4666591.5600|
|        COBBLE HILL|   3408766.7119|
| WILLIAMSBURG-SOUTH|   5429498.6165|
|   BROOKLYN-UNKNOWN|  11832012.2727|
|       CONEY ISLAND|  11419573.0952|
|      EAST NEW YORK|   4399124.3795|
|            MADISON|   3714957.2174|
|           BUSHWICK|   4242596.1286|
|        FORT GREENE|   3808404.5049|
+-------------------+---------------+
only showing top 20 rows



3.	Выведите среднюю полную площадь жилья (gross_sqft) для всех сочетаний налоговых категорий (tax_class) и лет продажи (year_of_sale).

In [ ]:
data.groupBy('tax_class','year_of_sale').avg('gross_sqft').show()


+---------+------------+---------------+
|tax_class|year_of_sale|avg(gross_sqft)|
+---------+------------+---------------+
|       2B|        2010|      6967.1364|
|        4|        2009|     20391.4259|
|        2|        2017|      4425.7555|
|       2B|        2008|      6722.5676|
|       2A|        2007|      4175.5000|
|       1B|        2015|         0.0000|
|       2C|        2012|       176.1364|
|       2C|        2017|         0.0000|
|       2B|        2003|      8349.0769|
|       1A|        2016|         0.0000|
|       1C|        2007|         0.0000|
|     null|        2003|         0.0000|
|     null|        2007|         0.0000|
|        4|        2003|     32590.4783|
|       2C|        2003|         0.0000|
|        2|        2003|     33540.0611|
|       2A|        2010|      3993.1702|
|       1A|        2003|         0.0000|
|        3|        2011|         0.0000|
|     null|        2015|         0.0000|
+---------+------------+---------------+
only showing top

4.	Выведите таблицу, содержащую количество пустых (null) значений для каждой колонки.

In [ ]:
from pyspark.sql.functions import isnan, when, count, col

data.select([count(when(col(c).isNull(), c)).alias(c) for c in data.columns]).show()


+---+--------+------------+-----------------------+---------+-----+---+--------+--------------+--------+----------------+--------+-----------------+----------------+-----------+---------+----------+----------+-----------------+----------------------+----------+---------+------------+---------+---+------+------+----------+-------+-------+--------+----------+----------+----------+---------+----------+--------+---------+---------+---------+---------+---------+--------+--------+-------+-------+-------+---------+---------+---------+-------+---------+---------+---------+-------+--------+-------+-------+----------+----------+----------+---------+----------+---------+----------+--------+---------+--------+----------+--------+--------+---------+---------+---+--------+----------+-------+--------+----------+---------+----------+---------+---------+----------+----------+--------+--------+--------+--------+-------+--------+--------+---+-------+---------+------+------+-------+------+-------+------+-